In [1]:
# 패키지 불러오기
# 출처 https://right1203.github.io/study/2019/02/28/lol-duo-tier/
import pickle # 리스트 안의 데이터프레임 형태 저장
import requests # api 요청
import json
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time

from skimage import io # 미니맵 처리
from sklearn.preprocessing import MinMaxScaler

In [2]:
%matplotlib inline
sns.set()

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
r = requests.get('https://ddragon.leagueoflegends.com/api/versions.json') # version data 확인
current_version = r.json()[0] # 가장 최신 버전 확인
current_version

'10.22.1'

In [6]:
# requests 함수로 chapms 데이터 받아오기 (가장 최신 버전)
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/champion.json'.format(current_version))
parsed_data = r.json() # 파싱
info_df = pd.DataFrame(parsed_data)
info_df.head()

,type,format,version,data
Aatrox,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Akali', 'key': '..."
Alistar,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Alistar', 'key':..."
Amumu,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Amumu', 'key': '..."


In [7]:
# champ_info_df의 data 값들을 데이터프레임으로 변환

# 데이터의 각 행을 시리즈 형태로 변환하여 딕셔너리에 추가
champ_dic = {}
for i, champ in enumerate(info_df.data):
    champ_dic[i] = pd.Series(champ)

# 데이터 프레임 변환 후 Transpose
champ_df = pd.DataFrame(champ_dic).T

# output : 챔피언 데이터 안에도 info와 stats가 딕셔너리 형태임
# 이 데이터들을 데이터프레임으로 변환하여 각 챔피언에 대한 변수로 추가해야 한다.

# champ_df의 info, stats의 데이터를 변수로 추가
champ_info_df = pd.DataFrame(dict(champ_df['info'])).T
champ_stats_df = pd.DataFrame(dict(champ_df['stats'])).T

# 데이터 합치기
champ_df = pd.concat([champ_df, champ_info_df], axis=1)
champ_df = pd.concat([champ_df, champ_stats_df], axis=1)
# 이번 분석에서 필요없는 데이터 제거
champ_df = champ_df.drop(['version', 'image', 'info', 'stats'], axis=1)
champ_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 0 to 151
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    152 non-null    object 
 1   key                   152 non-null    object 
 2   name                  152 non-null    object 
 3   title                 152 non-null    object 
 4   blurb                 152 non-null    object 
 5   tags                  152 non-null    object 
 6   partype               152 non-null    object 
 7   attack                152 non-null    int64  
 8   defense               152 non-null    int64  
 9   magic                 152 non-null    int64  
 10  difficulty            152 non-null    int64  
 11  hp                    152 non-null    float64
 12  hpperlevel            152 non-null    float64
 13  mp                    152 non-null    float64
 14  mpperlevel            152 non-null    float64
 15  movespeed             1

In [8]:
champ_df.head()

,id,key,name,title,blurb,tags,partype,attack,defense,magic,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,Aatrox,266,아트록스,다르킨의 검,한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공...,"[Fighter, Tank]",피의 샘,8,4,3,...,3.0,1.00,0.000,0.000,0.0,0.0,60.00,5.00,2.500,0.651
1,Ahri,103,아리,구미호,룬테라가 가진 잠재력과 선천적으로 연결된 아리는 마법을 순수한 에너지의 구체로 변환...,"[Mage, Assassin]",마나,3,4,8,...,5.5,0.60,8.000,0.800,0.0,0.0,53.04,3.00,2.000,0.668
2,Akali,84,아칼리,섬기는 이 없는 암살자,킨코우 결사단과 그림자의 권이라는 지위를 버린 아칼리는 아이오니아인들에게 필요한 강...,[Assassin],기력,5,3,8,...,8.0,0.50,50.000,0.000,0.0,0.0,62.40,3.30,3.200,0.625
3,Alistar,12,알리스타,미노타우로스,비할 데 없이 강력한 전사라는 평판이 자자한 알리스타는 녹서스 제국에게 부족 전체가...,"[Tank, Support]",마나,6,9,5,...,8.5,0.85,8.500,0.800,0.0,0.0,62.00,3.75,2.125,0.625
4,Amumu,32,아무무,슬픈 미라,뼛속까지 사무치는 외로움과 쓸쓸함 속에 살아가는 아무무. 평생을 오로지 마음을 나눌...,"[Tank, Mage]",마나,2,6,8,...,9.0,0.85,7.382,0.525,0.0,0.0,53.38,3.80,2.180,0.736


In [9]:
# API에 Key를 매일 갱신하여 받아와야한다.
# API의 요청 제한은 2분에 100개인 듯하다. 언제 다 받아서 분석하지...

# api 요청

api_url_grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_url_master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
# 그랜드마스터, 마스터 데이터 가져와서 데이터프레임으로 변환
r = requests.get(api_url_grandmaster)
league_df = pd.DataFrame(r.json())
r = requests.get(api_url_master)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)

# entries 데이터프레임으로 변환하여 추가
league_df.reset_index(inplace=True)
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1)

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3331 entries, 0 to 3330
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          3331 non-null   object
 1   summonerId    3331 non-null   object
 2   summonerName  3331 non-null   object
 3   leaguePoints  3331 non-null   object
 4   wins          3331 non-null   object
 5   losses        3331 non-null   object
 6   veteran       3331 non-null   object
 7   inactive      3331 non-null   object
 8   freshBlood    3331 non-null   object
 9   hotStreak     3331 non-null   object
dtypes: object(10)
memory usage: 260.4+ KB


In [12]:
league_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,GRANDMASTER,LzaVwiWI2epzjMwhxTQWdnqm5UmZpw2VL8uUBxyBRoQx6AQ,상록중,465,859,836,False,False,True,False
1,GRANDMASTER,ZzzraaXCvB-JC0jUXCO1OiCrc70fbFAenyaEZV6GXbs6Ris,어린아이,429,235,146,False,False,True,False
2,GRANDMASTER,0P9fw5GWtwvzaCPtyYn9G_A69h0ivE3v_k0YTzdQtHBFdD0,달인 카직스,619,190,180,False,False,False,False
3,GRANDMASTER,ZFzFylvunoq_skWGIrE26nR5fmJzJUXR4azrP4bTDZLhjpg,Miss Jennie,474,982,934,False,False,False,False
4,GRANDMASTER,weMOuSbzX7lIbjHIuzQKPeiCFhS3FuzWCgj6dy7QnIF-b6I,HUYATV Osu,729,539,433,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...
3326,MASTER,aXc778IHXNgsfEXehMPf-3n4CcQB_V63RrWLpmJ7HNW2_pE,TES XianQun,123,223,211,False,False,False,False
3327,MASTER,qWHsZVs1wFGBQdPkIptS3lFbtWwqkmAPs6jxM5OGhE-6Ixc,DWG BervL,89,823,810,False,False,False,False
3328,MASTER,BHYWJYHnqj7z0N0gN8XxhSXLxkrjizcfZtu7DzATUifKU6o,파이크노벤이여,74,427,395,False,False,False,False
3329,MASTER,lVKsxroMr42EkPRBvxmSKuhLKK0ENiJeUssuaUhbUaX83QQ,For The KlNG,120,524,409,False,False,True,False


In [ ]:
# summonerId를 이용하여 accountId를 모두 받아와야 한다. 요청 제한으로 인해 오래 걸리므로 파일로 저장하여 관리
# API 요청 제한 : 1초에 20, 2분에 100

league_df['account_id'] = np.nan # account_id 초기화
for i, summoner_id in enumerate(league_df['summonerId']):
     # 각 소환사의 SummonerId와 API Key를 포함한 url을 만들고, Summoner API에서 AccountId를 가져와 채워넣는다.
     api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
     r = requests.get(api_url)
     while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
         time.sleep(5)
         r = requests.get(api_url)
     account_id = r.json()['accountId']
     league_df.iloc[i, -1] = account_id

league_df.to_csv('LeagueData.csv')

league_df = pd.read_csv('LeagueData.csv')
league_df.head()

In [ ]:
for i, summoner_id in enumerate(league_df['summonerId']):
    if i>=1300 and i%3 == 0: #3배수
         # 각 소환사의 SummonerId와 API Key를 포함한 url을 만들고, Summoner API에서 AccountId를 가져와 채워넣는다.
         api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
         r = requests.get(api_url)
         while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
             time.sleep(5)
             r = requests.get(api_url)
         account_id = r.json()['accountId']
         league_df.iloc[i, -1] = account_id
    elif i>=1300 and i%3 ==1: #나머지 1
        api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key_2
        r = requests.get(api_url)
        while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
             time.sleep(5)
             r = requests.get(api_url)
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
    elif i>=1300 and i%3 ==2:
        api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key_3
        r = requests.get(api_url)
        while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
             time.sleep(5)
             r = requests.get(api_url)
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
        
        


In [32]:
league_df_crawled = league_df[league_df['account_id'].isna()]

In [33]:
league_df_crawled.to_csv('LeagueData_0.csv')

In [34]:
league_df_uncrawled = league_df[league_df['account_id'].isna()]

In [35]:
league_df_uncrawled.to_csv('LeagueData_0_uncralwed.csv')

In [36]:
league_df_uncrawled = pd.read_csv('LeagueData_0_uncralwed.csv')

for i, summoner_id in enumerate(league_df_uncrawled['summonerId']):
    api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    account_id = r.json()['accountId']
    league_df_uncrawled.iloc[i, -1] = account_id
   
        

In [37]:
league_df_uncrawled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2031 non-null   int64 
 1   tier          2031 non-null   object
 2   summonerId    2031 non-null   object
 3   summonerName  2031 non-null   object
 4   leaguePoints  2031 non-null   int64 
 5   wins          2031 non-null   int64 
 6   losses        2031 non-null   int64 
 7   veteran       2031 non-null   bool  
 8   inactive      2031 non-null   bool  
 9   freshBlood    2031 non-null   bool  
 10  hotStreak     2031 non-null   bool  
 11  account_id    2031 non-null   object
dtypes: bool(4), int64(4), object(4)
memory usage: 135.0+ KB


In [45]:
league_df = pd.concat((league_df_crawled, league_df_uncrawled), axis=0)

In [47]:
league_df.index = range(len(league_df))

In [49]:
league_df.columns

Index(['tier', 'summonerId', 'summonerName', 'leaguePoints', 'wins', 'losses',
       'veteran', 'inactive', 'freshBlood', 'hotStreak', 'account_id',
       'Unnamed: 0'],
      dtype='object')

In [50]:
league_df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [51]:
league_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,account_id
0,GRANDMASTER,LzaVwiWI2epzjMwhxTQWdnqm5UmZpw2VL8uUBxyBRoQx6AQ,상록중,465,859,836,False,False,True,False,rmENtbyddaGgLtLktYhSd3USqSZMPGAkIugqWQD3IqtRmjM
1,GRANDMASTER,ZzzraaXCvB-JC0jUXCO1OiCrc70fbFAenyaEZV6GXbs6Ris,어린아이,429,235,146,False,False,True,False,d6uQQ7dhZWowIrdyuzNGYwfOmKfBBzJdXmYO8uRfgK4o4E...
2,GRANDMASTER,0P9fw5GWtwvzaCPtyYn9G_A69h0ivE3v_k0YTzdQtHBFdD0,달인 카직스,619,190,180,False,False,False,False,-HspVEecUacWVy0K2mJdM0_TpHVDAzFTZUi51jhLGdljzN4
3,GRANDMASTER,ZFzFylvunoq_skWGIrE26nR5fmJzJUXR4azrP4bTDZLhjpg,Miss Jennie,474,982,934,False,False,False,False,mFGjmed2lW-7piD3Q6gx2BPz2j8Jd6nR-1wB0VnvtVxBTV...
4,GRANDMASTER,weMOuSbzX7lIbjHIuzQKPeiCFhS3FuzWCgj6dy7QnIF-b6I,HUYATV Osu,729,539,433,True,False,False,True,daOXRy7Jw_6tzppmctQf6Utie1XkYIQsnGBWY8ncA3Gzev...
...,...,...,...,...,...,...,...,...,...,...,...
3326,MASTER,aXc778IHXNgsfEXehMPf-3n4CcQB_V63RrWLpmJ7HNW2_pE,TES XianQun,123,223,211,False,False,False,False,XA6p6L1JRVWB2U8tmzrKMpuB0knTj6LzixsgVQEAc3v_FB...
3327,MASTER,qWHsZVs1wFGBQdPkIptS3lFbtWwqkmAPs6jxM5OGhE-6Ixc,DWG BervL,89,823,810,False,False,False,False,JR32Es6GV4NDhSOBvOoJlZQ8yeWiFgz_UPdry1g9XaZ4j1s
3328,MASTER,BHYWJYHnqj7z0N0gN8XxhSXLxkrjizcfZtu7DzATUifKU6o,파이크노벤이여,74,427,395,False,False,False,False,rc2QaJrbRkjyomSDYgXECtArRe7Ioslnz8IXoWJHkdG7v_4
3329,MASTER,lVKsxroMr42EkPRBvxmSKuhLKK0ENiJeUssuaUhbUaX83QQ,For The KlNG,120,524,409,False,False,True,False,CgnmleABD_91KstWVZXm-RKL9Fb8Yhv9sAFdAB42pi_fP40


In [52]:
league_df.to_csv('LOL_User_IDs.csv')

In [64]:
# Match info 데이터 받기 (gameId를 얻기 위해서)

season = str(13)

for account_id in league_df['account_id'][100:]:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + \
                  '?season=' + season + '&api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    print('R ', end="") #읽을 때마다 가져오기
match_info_df.to_csv('MatchInfoData.csv')

# 저장된 데이터 받아오기
# match_info_df = pd.read_csv('MatchInfoData.csv', index_col=0)
# match_info_df.reset_index(inplace=True)

R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R R 

In [82]:
match_info_df.sample(10)

,index,platformId,gameId,champion,queue,season,timestamp,role,lane
103340,40,KR,4726142199,22,700,13,1603020910730,DUO_CARRY,BOTTOM
72726,26,KR,4760231173,76,420,13,1604416520543,NONE,JUNGLE
35582,82,KR,4751326561,21,420,13,1604072387599,DUO_CARRY,BOTTOM
190437,69,KR,4742162586,64,420,13,1603690666101,NONE,JUNGLE
135992,9,KR,4743043006,103,420,13,1603714785601,SOLO,MID
66694,94,KR,4672240161,84,420,13,1601061282006,SOLO,MID
195296,28,KR,4751046574,202,420,13,1604067182735,DUO_SUPPORT,NONE
176454,86,KR,4743756531,39,420,13,1603752034374,DUO,BOTTOM
124661,61,KR,4751853409,22,420,13,1604084678267,DUO,BOTTOM
162267,85,KR,4734029922,14,420,13,1603372233591,SOLO,TOP


In [67]:
len(match_info_df)

333051

In [68]:
r = requests.get('https://ddragon.leagueoflegends.com/api/versions.json') # version data 확인
current_version = r.json()[0] # 가장 최신 버전 확인
current_version

'10.22.1'

In [69]:
match_info_df = pd.read_csv('MatchInfoData.csv', index_col=0)
match_info_df.reset_index(inplace=True)

In [84]:
temp_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4726142199,KR,1603020910730,1744,700,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [95]:
match_gameid = match_info_df.gameId.drop_duplicates()

In [97]:
match_gameid.index = range(len(match_gameid))

In [101]:
len(match_gameid)

174948

In [132]:
match_df0 = pd.DataFrame()
cnt =0
for game_id in match_gameid[70000:110000]: # 이전의 매치에 대한 정보 데이터에서 게임 아이디를 가져온다
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T # 게임 아이디에 대한 매치 데이터를 받아서 추가
    trueset = [ #게임 가져올 조건 지정.
               temp_df.gameMode[0] == 'CLASSIC', #클래식일 때만
               temp_df.gameVersion[0][0:5] in ['10.20', '10.21', '10.22'], #최신 패치만 가져오기
               temp_df.gameType[0] == 'MATCHED_GAME', #gametype이 Matched-game일때만
               temp_df.gameDuration[0]>=300, #게임 시간 5분 이상
               temp_df.queueId[0] in [420, 430, 440]
              ] 
    if all(trueset): #조건 만족할 때만 가져오기
        temp_df = temp_df.drop(columns=['platformId', 'mapId', 'seasonId', 'gameMode', 'gameType' ], axis=1) 
        #우선 불필요한 정보는 드롭하고... 합치기
        match_df0 = pd.concat([match_df0, temp_df])
        cnt +=1
    if cnt%1000 == 0:
        print(f'데이터 {cnt}개 크롤링 완료!')

데이터 1000개 크롤링 완료!
데이터 2000개 크롤링 완료!
데이터 3000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 4000개 크롤링 완료!
데이터 5000개 크롤링 완료!
데이터 6000개 크롤링 완료!
데이터 7000개 크롤링 완료!
데이터 8000개 크롤링 완료!
데이터 9000개 크롤링 완료!
데이터 10000개 크롤링 완료!
데이터 11000개 크롤링 완료!
데이터 12000개 크롤링 완료!
데이터 13000개 크롤링 완료!
데이터 14000개 크롤링 완료!
데이터 15000개 크롤링 완료!
데이터 16000개 크롤링 완료!
데이터 17000개 크롤링 완료!
데이터 18000개 크롤링 완료!
데이터 19000개 크롤링 완료!
데이터 20000개 크롤링 완료!
데이터 21000개 크롤링 완료!
데이터 22000개 크롤링 완료!
데이터 23000개 크롤링 완료!


In [133]:
match_df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23016 entries, 0 to 0
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   gameId                 23016 non-null  object
 1   gameCreation           23016 non-null  object
 2   gameDuration           23016 non-null  object
 3   queueId                23016 non-null  object
 4   gameVersion            23016 non-null  object
 5   teams                  23016 non-null  object
 6   participants           23016 non-null  object
 7   participantIdentities  23016 non-null  object
dtypes: object(8)
memory usage: 1.6+ MB


In [136]:
match_df0.to_pickle(f'Gameinfo_70000_to_110000.pickle')

In [134]:
vax = list(match_gameid).index(match_df0.iloc[-1,0])

In [135]:
vax

109994